In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm,t,uniform,f
from sklearn.linear_model import LinearRegression
import termcolor
from numba import jit

def rprint(string, colors='green'):
    print(termcolor.colored(string, color=colors))

In [2]:
def RSS(X, model,Y):
    ret=0
    return (X@model-Y).T@(X@model-Y)


def TSS(X,Y):
    return ret


In [3]:
Y=np.array([83,85,
           84,85,85,86,86,87,
           86,87,87,87,88,88,88,88,88,89,90,
           89,90,90,91,
           90,92])
X=np.zeros((len(Y),5))
X[0:2,0]=1
X[2:8,1]=1
X[8:19,2]=1
X[19:23,3]=1
X[23:25,4]=1
X,Y

(array([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.]]),
 array([83, 85, 84, 85, 85, 86, 86, 87, 86, 87, 87, 87, 88, 88, 88, 88, 88,
        89, 90, 89, 90, 90, 91, 90, 92]))

In [4]:
F=X.T@X
F_1=np.linalg.inv(F)
F.diagonal(),F_1.diagonal()

(array([ 2.,  6., 11.,  4.,  2.]),
 array([0.5       , 0.16666667, 0.09090909, 0.25      , 0.5       ]))

In [5]:
model=F_1@X.T@Y
model

array([84.        , 85.5       , 87.81818182, 90.        , 91.        ])

In [9]:
Rss=RSS(X, model, Y)
print(f"{Rss=}")
pvals=[]
for i in range(5):
    for j in range(i+1,5):
        delta = (model[j]-model[i])/(Rss*(F_1[i,i]+F_1[j,j]))**0.5*(len(Y)-5)**0.5
        pvals.append(t.sf(delta,len(Y)-5))
        colors= "green" if 0.05>pvals[-1] else "red"
        rprint(f"  B_{i+1}==B_{j+1} pval = {pvals[-1]}",colors)
        rprint(f"\t\t{delta=:.2f}\n",colors)

Rss=np.float64(23.13636363636364)
  B_1==B_2 pval = 0.05155020445041343
		delta=1.71

  B_1==B_3 pval = 8.310072438777822e-05
		delta=4.62

  B_1==B_4 pval = 1.3910557503277491e-06
		delta=6.44

  B_1==B_5 pval = 1.2062851384167881e-06
		delta=6.51

  B_2==B_3 pval = 0.0001975200501670035
		delta=4.25

  B_2==B_4 pval = 1.2767362453664136e-06
		delta=6.48

  B_2==B_5 pval = 2.042591552504244e-06
		delta=6.26

  B_3==B_4 pval = 0.001196669352181163
		delta=3.47

  B_3==B_5 pval = 0.000501281453002286
		delta=3.85

  B_4==B_5 pval = 0.1478956761373736
		delta=1.07



In [14]:
pvals=sorted(pvals)
print(*[round(i,7) for i in pvals])
print("(pval отсортирован)")
for i in range(10,0,-1):
    rprint(f"alf/{i}\t = {0.05/i:.4f}\n\
pval[{10-i+1:2d}] = {pvals[10-i]:.6f} =>{" не" if 0.05/i>pvals[10-i] else ""} \"выкидываем\"\n",\
          colors= "green" if 0.05/i>pvals[10-i] else "red")

1.2e-06 1.3e-06 1.4e-06 2e-06 8.31e-05 0.0001975 0.0005013 0.0011967 0.0515502 0.1478957
(pval отсортирован)
alf/10	 = 0.0050
pval[ 1] = 0.000001 => не "выкидываем"

alf/9	 = 0.0056
pval[ 2] = 0.000001 => не "выкидываем"

alf/8	 = 0.0063
pval[ 3] = 0.000001 => не "выкидываем"

alf/7	 = 0.0071
pval[ 4] = 0.000002 => не "выкидываем"

alf/6	 = 0.0083
pval[ 5] = 0.000083 => не "выкидываем"

alf/5	 = 0.0100
pval[ 6] = 0.000198 => не "выкидываем"

alf/4	 = 0.0125
pval[ 7] = 0.000501 => не "выкидываем"

alf/3	 = 0.0167
pval[ 8] = 0.001197 => не "выкидываем"

alf/2	 = 0.0250
pval[ 9] = 0.051550 => "выкидываем"

alf/1	 = 0.0500
pval[10] = 0.147896 => "выкидываем"

